# LangChain multi-doc retriever with ChromaDB

***New Points***
- Multiple Files
- ChromaDB
- Source info
- gpt-3.5-turbo API

## Setting up LangChain


In [13]:
import os
import dotenv
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import PyPDFLoader

In [14]:
DOT_ENV_PATH = "/Users/danielgeorge/Documents/work/ml/hypolab/Synapse/backend/node/.env"
dotenv.load_dotenv(DOT_ENV_PATH)

True

## Load multiple and process documents

In [15]:
# Load and process the text files
# loader = TextLoader('single_text_file.txt')
loader = DirectoryLoader('./data', glob="./*.pdf", loader_cls=PyPDFLoader)

documents = loader.load()

In [16]:
documents[0]

Document(page_content='0123456789();: Artificial intelligence (AI) has been called \na revolutionary tool for science1,2 and \nit has been predicted to play a creative \nrole in research in the future3. In the \ncontext of theoretical chemistry, for example, it is believed that AI can help \nsolve problems “in a way such that the human cannot distinguish between this [AI] and communicating with a human \nexpert”\n4. However, this excitement has not \nbeen shared by all scientists. Some have questioned whether advanced computational \napproaches can go beyond ‘numerics’\n5–9 and \ncontribute on a fundamental level to gaining \nof new scientific understanding10–12.\nIn this Perspective, we discuss how \nadvanced computational systems, and AI \nin particular, can contribute to scientific \nunderstanding: we overview what is \ncurrently possible and what might lie ahead. In addition to the review of the literature, we surveyed dozens of scientists working at \nthe interface of biology, che

In [17]:
#splitting the text into
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)

In [18]:
len(texts)

164

In [19]:
texts[3]

Document(page_content='language understanding in AI and related topics). Many other works contribute to related questions and should be mentioned \nhere. One important field of research in  \nAI is explainable AI, which aims to \ninterpret and explain how advanced AI algorithms come up with their solutions; \nsee, for instance, \nrefs.15–18. Whereas it is not \nnecessary, and we believe also not sufficient, to interpret the internal workings of the \nAI to get new scientific understanding, many of these tools and techniques can be very useful. We will briefly explain \nthem below with concrete examples in the \nnatural sciences. AI pioneer Donald Michie classified machine learning (ML) into three classes: weak, strong and ultrastrong, in \nwhich ultrastrong requires the machine to \nteach the human\n19. The ultrastrong ML is', metadata={'source': 'data/Krenn et al. - 2022 - On scientific understanding with artificial intelligence  Nature Reviews Physics.pdf', 'page': 0})

## create the DB

In [36]:
# Embed and store the texts
# Supplying a persist_directory will store the embeddings on disk
persist_directory = 'db'

## here we are using OpenAI embeddings but in future we will swap out to local embeddings
embedding = OpenAIEmbeddings()

vectordb = Chroma.from_documents(documents=texts,
                                 embedding=embedding,
                                 persist_directory=persist_directory)

In [10]:
# persiste the db to disk
vectordb.persist()
vectordb = None

In [11]:
# Now we can load the persisted database from disk, and use it as normal.
vectordb = Chroma(persist_directory=persist_directory,
                  embedding_function=embedding)

## Make a retriever

In [12]:
retriever = vectordb.as_retriever()

In [13]:
docs = retriever.get_relevant_documents("How would we use AAV to edit cells?")

In [14]:
len(docs)

4

In [15]:
retriever = vectordb.as_retriever(search_kwargs={"k": 2})

In [16]:
retriever.search_type

'similarity'

In [17]:
retriever.search_kwargs

{'k': 2}

## Make a chain

In [ ]:
# create the chain to answer questions
qa_chain = RetrievalQA.from_chain_type(llm=OpenAI(),
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True)

In [ ]:
## Cite sources
def process_llm_response(llm_response):
    print(llm_response['result'])
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

In [ ]:
# full example
query = "How would we use AAV to edit cells?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 AAV can be used to edit cells through a gene therapy or a passive vaccine. It can be produced in HEK293 cells using a platform of transient transfection or using baculovirus infection of Spodoptera frugiperda insect cells.


Sources:
data/data/Rumachik et al. - 2020 - Methods Matter Standard Production Platforms for.pdf
data/data/Rumachik et al. - 2020 - Methods Matter Standard Production Platforms for.pdf


In [ ]:
# break it down
query = """Favorite representations
- notations (Leibniz), automata, graphs (Bret victor), car recliner button
- keep reading design of everyday things
- Going from 1 to 0 registers through analogy (Python)
- Mendeleev and the periodic table
"""
llm_response = qa_chain(query)
# process_llm_response(llm_response)
llm_response

{'query': 'Favorite representations\n- notations (Leibniz), automata, graphs (Bret victor), car recliner button\n- keep reading design of everyday things\n- Going from 1 to 0 registers through analogy (Python)\n- Mendeleev and the periodic table\n',
 'result': " I don't know.",
 'source_documents': [Document(page_content='during the day. If yesterday you met three new people, and you \nwere made aware of the fact today, you might feel pressure d to \nmeet or exceed yesterday\'s number. If you were not keeping track \nof the daily number, yesterday\'s achievement would have no \npositive bearing on your actions today. Effectively this means that \neven if the artifacts we design for augmenting aspects of cognition \ndo not fun ction perfectly, we may get at least an initial \nimprovement in  functionality purely based on this measurement \nand increased awa reness phenomenon.  \nPopulations  \nUseful parallels with the biological sciences need not end with co -\nevolution. In his 1962 p

In [ ]:
query = "Who led the round in Pando?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 Iron Pillar and Uncorrelated Ventures.


Sources:
new_articles/05-03-ai-powered-supply-chain-startup-pando-lands-30m-investment.txt
new_articles/05-03-ai-powered-supply-chain-startup-pando-lands-30m-investment.txt


In [ ]:
query = "What did databricks acquire?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 Databricks acquired Okera, a data governance platform with a focus on AI.


Sources:
new_articles/05-03-databricks-acquires-ai-centric-data-governance-platform-okera.txt
new_articles/05-03-databricks-acquires-ai-centric-data-governance-platform-okera.txt


In [ ]:
query = "What is generative ai?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 Generative AI is a type of artificial intelligence that is used to create new content associated with a company, such as content for a website or ads. It can also be used to automate processes and workflows.


Sources:
new_articles/05-04-slack-updates-aim-to-put-ai-at-the-center-of-the-user-experience.txt
new_articles/05-03-nova-is-building-guardrails-for-generative-ai-content-to-protect-brand-integrity.txt


In [ ]:
query = "Who is CMA?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 The CMA stands for the Competition and Markets Authority.


Sources:
new_articles/05-04-cma-generative-ai-review.txt
new_articles/05-04-cma-generative-ai-review.txt


In [ ]:
qa_chain.retriever.search_type , qa_chain.retriever.vectorstore

('similarity', <langchain.vectorstores.chroma.Chroma at 0x7f9f7dc82aa0>)

In [ ]:
print(qa_chain.combine_documents_chain.llm_chain.prompt.template)

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Helpful Answer:


## Deleteing the DB

In [ ]:
!zip -r db.zip ./db

  adding: db/ (stored 0%)
  adding: db/chroma-collections.parquet (deflated 50%)
  adding: db/index/ (stored 0%)
  adding: db/index/index_metadata_59c51927-205d-4fd7-88d8-c7ba851bd2a5.pkl (deflated 5%)
  adding: db/index/uuid_to_id_59c51927-205d-4fd7-88d8-c7ba851bd2a5.pkl (deflated 39%)
  adding: db/index/index_59c51927-205d-4fd7-88d8-c7ba851bd2a5.bin (deflated 17%)
  adding: db/index/id_to_uuid_59c51927-205d-4fd7-88d8-c7ba851bd2a5.pkl (deflated 35%)
  adding: db/chroma-embeddings.parquet (deflated 29%)


In [ ]:
# To cleanup, you can delete the collection
vectordb.delete_collection()
vectordb.persist()

# delete the directory
!rm -rf db/

## Starting again loading the db

restart the runtime

In [ ]:
!unzip db.zip

Archive:  db.zip
   creating: db/
  inflating: db/chroma-collections.parquet  
   creating: db/index/
  inflating: db/index/index_metadata_59c51927-205d-4fd7-88d8-c7ba851bd2a5.pkl  
  inflating: db/index/uuid_to_id_59c51927-205d-4fd7-88d8-c7ba851bd2a5.pkl  
  inflating: db/index/index_59c51927-205d-4fd7-88d8-c7ba851bd2a5.bin  
  inflating: db/index/id_to_uuid_59c51927-205d-4fd7-88d8-c7ba851bd2a5.pkl  
  inflating: db/chroma-embeddings.parquet  


In [ ]:
import os

os.environ["OPENAI_API_KEY"] = ""

In [ ]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings

from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

In [ ]:
persist_directory = 'db'
embedding = OpenAIEmbeddings()

vectordb2 = Chroma(persist_directory=persist_directory,
                  embedding_function=embedding,
                   )

retriever = vectordb2.as_retriever(search_kwargs={"k": 2})

In [ ]:
# Set up the turbo LLM
turbo_llm = ChatOpenAI(
    temperature=0,
    model_name='gpt-3.5-turbo'
)

In [ ]:
# create the chain to answer questions
qa_chain = RetrievalQA.from_chain_type(llm=turbo_llm,
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True)

In [ ]:
## Cite sources
def process_llm_response(llm_response):
    print(llm_response['result'])
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

In [ ]:
# full example
query = "How much money did Pando raise?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

Pando raised $30 million in a Series B round, bringing its total raised to $45 million.


Sources:
new_articles/05-03-ai-powered-supply-chain-startup-pando-lands-30m-investment.txt
new_articles/05-03-ai-powered-supply-chain-startup-pando-lands-30m-investment.txt


### Chat prompts

In [ ]:
print(qa_chain.combine_documents_chain.llm_chain.prompt.messages[0].prompt.template)

Use the following pieces of context to answer the users question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
{context}


In [ ]:
print(qa_chain.combine_documents_chain.llm_chain.prompt.messages[1].prompt.template)

{question}


In [42]:
try:
    with open('/Users/danielgeorge/Documents/work/ml/hypolab/Synapse/server/dirty_index/test.txt', 'w') as f:
        f.write('test')
except Exception as e:
    print(e)

[Errno 2] No such file or directory: '/Users/danielgeorge/Documents/work/ml/hypolab/Synapse/server/dirty_index/test.txt'
